In [1]:
import sys
sys.path.append("./expectation_value")

from expectationvalue import ExpVal
import numpy as np
import qiskit.quantum_info as qif
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter
from tqdm import tqdm


In [2]:
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver


class MoleculeHamiltonian():
    """
    The MoleculeHamiltonian class uses qiskit_nature functions to easly obtain the qubit hamiltonian and
    groundstate/ground energy for a given molecule.
    
    """
    
    def __init__(self,
                 molecule,
                 converter = QubitConverter(JordanWignerMapper()),
                 basis = 'sto3g',
                ):
        """

        Parameters
        ----------
        molecule : string.
            Molecule geometry.
        converter : QubitConverter.
            Convert which transforms the second quantized hamiltonian to a qubit hamiltonian. By default,
            we use the JordanWignerMapper without reductions.
        basis : string.
            Basis set name as recognized by PySCF. We use sto3g as default, but others can be chosen
            from the list of PySCF’s valid basis set names 1_.
        
        . _1: https://pyscf.org/pyscf_api_docs/pyscf.gto.basis.html#module-pyscf.gto.basis
        """
        
        self.molecule = molecule
        self.basis = basis
        driver = PySCFDriver(
            atom= self.molecule,
            basis=self.basis,
            charge=0,
            spin=0,
            unit=DistanceUnit.ANGSTROM,
            )                                  # Define the PySCFDriver for a given molecule
        
        self.driver = driver
        self.problem = driver.run()            # Define the ElectronicStructureProblem from the driver
        self.converter = converter
        
    def Hamiltonian(self,
                    freeze_core = False,
                    remove_orbitals = None):
        
        
        """
        Returns the qubit hamiltonian, allowing Active-Space reduction using the FreezeCoreTransformer to
        obtain a hamiltonian with a lower number of qubits.


        Parameters
        ----------
        freeze_core : bool.
            Choose to inactive and remove the core orbitals according to count_core_orbitals.
        remove_orbitals : List, optional.
            Choose to remove aditional orbitals.
            
        Returns
        -------
        qubit_op : PauliSumOp.
            Qubit hamiltonian.
        """
        problem = self.problem
        fermionic_op = self.problem.hamiltonian.second_q_op()
            
        if freeze_core or remove_orbitals is not None:
            fc_transformer = FreezeCoreTransformer(freeze_core=freeze_core, remove_orbitals=remove_orbitals)
            problem = fc_transformer.transform(problem)
            fermionic_op = problem.hamiltonian.second_q_op()
            
        qubit_op = self.converter.convert(fermionic_op,
                                           sector_locator=problem.symmetry_sector_locator)
        return qubit_op
    
    def ComputeGroundState(self , circuit = True):
        
        """
        Compute the exact ground state and ground state energy using the NumPyMinimumEigensolver.

        Parameters
        ----------
        circuit : bool.
            If True, returns a quantum circuit that constructs the ground state. If False,
            returns the ground state as an array using Statevector.


        Returns
        -------
        state : QuantumCircuit or array(2**num_qubits)
            Ground state as a quantum circuit or as an array.
        eigenvalue : float.
            Ground state energy.
        """
        solver = NumPyMinimumEigensolver()
        
        ground_solver = GroundStateEigensolver(self.converter, solver)
        ground_state = ground_solver.solve(self.problem)
        
        state = ground_state.groundstate[0]
        eigenvalue = ground_state.groundenergy
        
        if circuit:
            return state, eigenvalue
        else:
            return qif.Statevector(state).data, eigenvalue

In [3]:
def get_number_nbody_terms(hamiltonian):
    """
    Gets the needed data of the hamiltonian to compute the expectation values using ExpVal.exp_val().


    Parameters
    ----------
    hamiltonian : PauliSumOp.
        Hamiltonian.


    Returns
    -------
    bodies : list.
        N-bodies interactions.
    """
    
    n_bodies_inter = []
    
    for i in range(len(hamiltonian)):
        pauli_string = hamiltonian[i].to_pauli_op().primitive.to_label()
        num_x = pauli_string.count('X')
        num_y = pauli_string.count('Y')
        
        n_body_terms = num_x+num_y
        
        n_bodies_inter.append(n_body_terms)
    
    bodies = list(set(n_bodies_inter))
    
    return bodies


def get_obs_data(hamiltonian):
    """
    Gets the needed data of the hamiltonian to compute the expectation values using ExpVal.exp_val().


    Parameters
    ----------
    hamiltonian : PauliSumOp.
        Hamiltonian.


    Returns
    -------
    coeffs : array(num_paulis).
        Coefficients of each pauli string of the hamiltonian.
    obs : array(2, 2, n_qubits, num_paulis)
        Observable made of every pauli string of the hamiltonian.
    """
    
    num_paulis = len(hamiltonian)
    n_qubits = hamiltonian.num_qubits

    paulis_dict = {'X': np.array([[0., 1.], [1., 0.]], dtype="complex"),
               'Y': np.array([[0., -1.*1j], [1.*1j, 0.]], dtype="complex"),
               'Z': np.array([[1., 0], [0, -1.]], dtype="complex"),
               'I': np.array([[1., 0], [0, 1.]], dtype="complex")}

    obs = np.zeros((2, 2, n_qubits, num_paulis), dtype=complex)
    coeffs = np.zeros(num_paulis, dtype=complex)

    for i in range(1, num_paulis):   # OJO CON EL PRIMER TERMINO
        pauli_string_coeff = hamiltonian[i].coeffs[0]
        pauli_string = hamiltonian[i].to_pauli_op().primitive.to_label()
        pauli_string_list = [paulis_dict[i] for i in pauli_string]
        obs[:,:,:,i] =  np.stack(pauli_string_list, axis=-1)
        coeffs[i] = pauli_string_coeff
    
    return coeffs, obs

In [4]:
#########################################################################################################

In [5]:
dist = 1.0
molecules = {
    "BeH2": "Be .0 .0 .0; H .0 .0 -" + str(dist) + "; H .0 .0 " + str(dist),
    "H2":"H .0 .0 .0; H .0 .0 " + str(dist),
    "LiH":"Li .0 .0 .0; H .0 .0 " + str(dist),
    "H2O": "H -0.0399 -0.0038 0.0; O 1.5780 0.8540 0.0; H 2.7909 -0.5159 0.0",
    "NH3": "N 0.0 0.0 0.149; H 0.0 0.947 -0.348; H  0.821 -0.474 -0.348; H -0.821 -0.474 -0.348"
}

In [6]:
# mol = 'H2'
# mol = 'LiH'
# mol = 'BeH2'
mol = "H2O"
# mol = "NH3"

In [7]:
converter = QubitConverter( JordanWignerMapper(), z2symmetry_reduction=None)
molecular_hamiltonian = MoleculeHamiltonian(molecules[mol] , converter=converter, basis='sto3g')
hamiltonian = molecular_hamiltonian.Hamiltonian( freeze_core = False )

driver = molecular_hamiltonian.driver
problem = molecular_hamiltonian.problem
n_qubits = hamiltonian.num_qubits

bodies = get_number_nbody_terms(hamiltonian)

In [8]:
circ_gs, eig_gs = molecular_hamiltonian.ComputeGroundState()
eig_gs

-79.59454251461254

In [9]:
total_shots = 6000000
r = 30
reps = 1

In [10]:
# algorithm = ExpVal(n_shots = total_shots-1000,
#         bodies = bodies, 
#         r = r, 
#         r_shots = 1000,
#         n_qubits = n_qubits)

# algorithm.get_interferences(circ_gs)

# coeffs, obs = get_obs_data(hamiltonian)

# exp_val = algorithm.exp_val(obs)


# results = np.sum(exp_val*coeffs).real

In [11]:
results_god = []

for i in tqdm(range(reps)):
    algorithm = ExpVal(n_shots = total_shots-total_shots//4,
            bodies = bodies, 
            r = r, 
            r_shots = total_shots//4,
            n_qubits = n_qubits)

    algorithm.get_shadows_R(circ_gs)

    coeffs, obs = get_obs_data(hamiltonian)

    exp_val = algorithm.exp_val(obs)

    first_term = hamiltonian[0].coeffs[0].real

    print('Exp val calculado: ', np.sum(exp_val*coeffs).real + first_term)
    results_god.append(np.sum(exp_val*coeffs).real + first_term)

100%|████████████████████████████████████████████| 1/1 [01:43<00:00, 103.64s/it]

Exp val calculado:  -79.67825421053438


In [12]:
print(eig_gs)

print(np.abs(results_god-eig_gs))

print(np.median(np.abs(results_god-eig_gs)))

-79.59454251461254
[0.0837117]
0.08371169592183492


In [13]:
results = []


for i in tqdm(range(reps)):
    algorithm = ExpVal(n_shots = total_shots-1000,
            bodies = bodies, 
            r = r, 
            r_shots = 1000,
            n_qubits = n_qubits)

    algorithm.get_interferences(circ_gs)

    coeffs, obs = get_obs_data(hamiltonian)

    exp_val = algorithm.exp_val(obs)
    
    first_term = hamiltonian[0].coeffs[0].real

    print('Exp val calculado: ', np.sum(exp_val*coeffs).real + first_term)
    results.append(np.sum(exp_val*coeffs).real + first_term)

100%|████████████████████████████████████████████| 1/1 [02:13<00:00, 133.19s/it]

Exp val calculado:  -79.58968993213763


In [14]:
print(eig_gs)

print(np.abs(results-eig_gs))

print(np.median(np.abs(results-eig_gs)))

-79.59454251461254
[0.00485258]
0.004852582474910605


In [15]:
num_paulis = len(hamiltonian)
shots_runtime = total_shots//num_paulis
shots_opflow = total_shots//num_paulis

In [16]:
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session, Options

#QiskitRuntimeService.save_account(channel="ibm_quantum", token="1a92dbd3ad8a3a75036e3b0fed7d2a10a0dfef2277960a619c033757aa1aa9bcc8f95922671a2fbd0b05339eea4aae6305bc43b5b2e6d33b161346f9983adf48")
service = QiskitRuntimeService(channel="ibm_quantum")

options = Options()
options.optimization_level = 1
options.resilience_level = 0
options.execution.shots = shots_runtime

# with Session(service=service, backend="ibmq_qasm_simulator") as session:
#     estimator = Estimator(session=session, options = options)

#     job = estimator.run(circ_gs, hamiltonian)

#     result = job.result()

# result_runtime = result.values[0]
# np.abs(result_runtime-eig_gs)

In [ ]:
results_runtime = []

for i in tqdm(range(rep)):


    with Session(service=service, backend="ibmq_qasm_simulator") as session:
        estimator = Estimator(session=session, options = options)

        job = estimator.run(circ_gs, hamiltonian)

        result = job.result()

    result_runtime = result.values[0]
    
    results_runtime.append(result_runtime)

 20%|█████████                                    | 1/5 [00:27<01:49, 27.32s/it]

In [ ]:
print(eig_gs)

print(np.abs(results_runtime-eig_gs))

print(np.median(np.abs(results_runtime-eig_gs)))

In [ ]:
# from qiskit.opflow import PauliExpectation, CircuitSampler, StateFn, CircuitStateFn
# from qiskit import Aer
# from qiskit.utils import QuantumInstance

# backend = Aer.get_backend("aer_simulator")
# quantum_instance = QuantumInstance(backend, shots=shots_opflow)

# op = hamiltonian
# psi = CircuitStateFn(circ_gs)

# measurable_expression = StateFn(op, is_measurement=True).compose(psi)
# expectation = PauliExpectation().convert(measurable_expression)  
# sampler = CircuitSampler(quantum_instance).convert(expectation)

# result_opflow = sampler.eval().real
# np.abs(result_opflow-eig_gs)